In [3]:
import uproot
import re
from collections import defaultdict

root_path = "/mnt/medley/LucasAnalysis/2023/XS_calcs/newresults/Fe_p_DDX.root"
file = uproot.open(root_path)

pattern = re.compile(r"p_ang_(\d+)_deg_(\d+\.?\d*)_(\d+\.?\d*)MeV_MatchC")

grouped_hists = defaultdict(list)
for key in file.keys():
    key_name = key.decode() if isinstance(key, bytes) else key
    match = pattern.match(key_name)
    if match:
        angle = int(match.group(1))
        e_min = match.group(2)
        e_max = match.group(3)
        energy_label = f"{e_min}_{e_max}"
        grouped_hists[energy_label].append((angle, key_name))


In [4]:
import uproot
import numpy as np
import matplotlib.pyplot as plt
import re
from collections import defaultdict

# Caminho para o arquivo ROOT
root_path = "/mnt/medley/LucasAnalysis/2023/XS_calcs/newresults/Fe_p_DDX.root"

# Abrir o arquivo ROOT
file = uproot.open(root_path)

# Coletar todos os nomes de objetos do tipo TH1D
hist_keys = [key for key in file.keys() if "MatchC" in key and "TH1D" in str(type(file[key].memberwise_properties))]

# Agrupar por faixa de energia usando regex
pattern = re.compile(r"p_ang_(\d+)_deg_(\d+\.?\d*)_(\d+\.?\d*)MeV_MatchC")
grouped_hists = defaultdict(list)

for key in hist_keys:
    match = pattern.match(key)
    if match:
        angle = int(match.group(1))
        e_min = match.group(2)
        e_max = match.group(3)
        energy_label = f"{e_min}_{e_max}"
        grouped_hists[energy_label].append((angle, key))

# Plotar os gráficos para cada faixa de energia
for energy_label, angle_keys in grouped_hists.items():
    # Ordenar por ângulo crescente
    angle_keys.sort()

    angles = [angle for angle, _ in angle_keys]
    colors = plt.cm.jet(np.linspace(0, 1, len(angles)))

    fig = plt.figure(figsize=(12, 10))
    ax = fig.add_subplot(projection='3d')

    for i, (angle, key) in enumerate(angle_keys):
        hist = file[key]
        x = hist.axis().centers()  # Bin centers (energia)
        y = hist.values()          # Conteúdo dos bins
        z = np.full_like(x, angle) # ângulo constante em Z

        ax.plot(x, z, y, color=colors[i], label=f"{angle}°")

    ax.set_xlabel('Energy (MeV)', labelpad=10, fontsize=14)
    ax.set_ylabel('Angle (deg)', labelpad=10, fontsize=14)
    ax.set_zlabel(r"$d^2\sigma / d\Omega dE$ (arb. units)", labelpad=10, fontsize=14)

    ax.tick_params(axis='x', labelsize=14)
    ax.tick_params(axis='y', labelsize=14)
    ax.tick_params(axis='z', labelsize=14)

    ax.text2D(0.3, 0.85, f"$^{{nat}}$Fe(n,xp), neutron E = {energy_label.replace('_', '–')} MeV",
              transform=ax.transAxes,
              fontsize=16, 
              bbox=dict(boxstyle="round,pad=0.4", facecolor="lightgray", edgecolor="black", alpha=0.7))

    ax.view_init(elev=30, azim=315)
    ax.set_box_aspect(aspect=None, zoom=0.85)

    plt.tight_layout()
    plt.savefig(f"Fe_spectra_3D_{energy_label}_MeV.png", dpi=300)
    plt.close()

print("✅ Plots salvos para cada faixa de energia.")


AttributeError: 'Model_TH1D_v3' object has no attribute 'memberwise_properties'

In [5]:
import uproot
import numpy as np
import matplotlib.pyplot as plt
import re
from collections import defaultdict

# Caminho para o arquivo ROOT
root_path = "/mnt/medley/LucasAnalysis/2023/XS_calcs/newresults/Fe_p_DDX.root"

# Abrir o arquivo ROOT
file = uproot.open(root_path)

# Coletar todos os nomes de objetos do tipo TH1D com "MatchC" no nome
hist_keys = [key for key in file.keys() if "MatchC" in key and file[key].classname == "TH1D"]

# Regex para extrair ângulo e intervalo de energia do nome do histograma
pattern = re.compile(r"p_ang_(\d+)_deg_(\d+\.?\d*)_(\d+\.?\d*)MeV_MatchC")

# Agrupar histogramas por faixa de energia
grouped_hists = defaultdict(list)
for key in hist_keys:
    match = pattern.match(key)
    if match:
        angle = int(match.group(1))
        e_min = match.group(2)
        e_max = match.group(3)
        energy_label = f"{e_min}_{e_max}"
        grouped_hists[energy_label].append((angle, key))

# Loop para plotar e salvar uma figura 3D para cada faixa de energia
for energy_label, angle_keys in grouped_hists.items():
    # Ordenar pela ordem dos ângulos
    angle_keys.sort()

    angles = [angle for angle, _ in angle_keys]
    colors = plt.cm.jet(np.linspace(0, 1, len(angles)))

    fig = plt.figure(figsize=(12, 10))
    ax = fig.add_subplot(projection='3d')

    for i, (angle, key) in enumerate(angle_keys):
        hist = file[key]
        x = hist.axis().centers()  # Centros dos bins (energia)
        y = hist.values()          # Conteúdo dos bins
        z = np.full_like(x, angle) # Ângulo constante para cada linha

        ax.plot(x, z, y, color=colors[i], label=f"{angle}°")

    ax.set_xlabel('Energy (MeV)', labelpad=10, fontsize=14)
    ax.set_ylabel('Angle (deg)', labelpad=10, fontsize=14)
    ax.set_zlabel(r"$d^2\sigma / d\Omega dE$ (arb. units)", labelpad=10, fontsize=14)

    ax.tick_params(axis='x', labelsize=14)
    ax.tick_params(axis='y', labelsize=14)
    ax.tick_params(axis='z', labelsize=14)

    ax.text2D(0.3, 0.85, f"$^{{nat}}$Fe(n,xp), neutron E = {energy_label.replace('_', '–')} MeV",
              transform=ax.transAxes,
              fontsize=16,
              bbox=dict(boxstyle="round,pad=0.4", facecolor="lightgray", edgecolor="black", alpha=0.7))

    ax.view_init(elev=30, azim=315)
    ax.set_box_aspect(aspect=None, zoom=0.85)

    plt.tight_layout()
    plt.savefig(f"Fe_spectra_3D_{energy_label}_MeV.png", dpi=300)
    plt.close()

print(f"✅ Plots gerados e salvos: {len(grouped_hists)} faixas de energia.")


✅ Plots gerados e salvos: 25 faixas de energia.
